In [9]:
from flytekit.sdk.types import Types
from flytekit.common import utils
from flytekit.common.workflow_execution import SdkWorkflowExecution

from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
wf = SdkWorkflowExecution.fetch(project="flytekubecondemo2019", domain="development", name="f65c48801eba846cf95f")


In [5]:
print(wf.outputs["trained_models"][0].uri)

s3://lyft-modelbuilder/h4/0d357758547f07aa61adcc980cd04bba


In [12]:
Image(url= "https://image.shutterstock.com/image-photo/beautiful-green-red-aurora-dancing-600w-1068442454.jpg", height=600)


In [ ]:
from flytekit.sdk.tasks import (
    python_task,
    inputs,
    outputs,
)
from flytekit.sdk.types import Types
from flytekit.common import utils

from demo_metrics.tasks.confusion_matrix import _plot_confusion_matrix as pm, _sample_train as st
import numpy as np

@inputs(y_true=[Types.Integer], y_pred=[Types.Integer], title=Types.String, normalize=Types.Boolean, classes=[Types.String])
@outputs(matrix=[[Types.Integer]], visual=Types.Blob)
@python_task
def confusion_matrix(wf_params, y_true, y_pred, title, normalize, classes, matrix, visual):
    with utils.AutoDeletingTempDir('test') as tmpdir:
        f_path = tmpdir.get_named_tempfile("visual")
        cm = pm(np.asarray(y_true), np.asarray(y_pred), classes=np.asarray(classes), title=title, normalize=normalize, to_file_path=f_path)
        visual.set(f_path)
        m = []
        for i in range(cm.shape[0]):
            m.append([])
            for j in range(cm.shape[1]):
              m[i].append(j)
        matrix.set(m)

cm = confusion_matrix.unit_test(y_true=y_test.tolist(), y_pred=y_pred.tolist(), title='Confusion matrix, without normalization', normalize=False, classes=class_names.tolist())